In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
team_stats = pd.read_csv('../../footy_data/championship_teams_2022.csv')
games = pd.read_csv('../../footy_data/championship_games_2022.csv')

In [3]:
games_for_later = games
#I am saving this new variable so it will be the original df when I want to include games that are 'incomplete' later

In [4]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [5]:
games = games.loc[games['status'] == 'complete']
#We only want games that have been completed

In [6]:
team_stats['avg_home_xg_for_home_team'] = ""
team_stats['avg_away_xg_against_home_team'] = ""
team_stats['avg_xg_for_home_team'] = ""
team_stats['avg_xg_against_home_team'] = ""
team_stats['avg_home_xg_for_away_team'] = ""
team_stats['avg_away_xg_against_away_team'] = ""
team_stats['avg_xg_for_away_team'] = ""
team_stats['avg_xg_against_away_team'] = ""
#Creating average xG and xGA for each team myself because FootyStats is too lazy to put it correctly in the spreadsheet

In [7]:
#CODE TO FIX XG AND XGA HERE, use numpy
#average of home_team = common_name[0]xgfor and away_team = common_name[0] xgfor
def get_avg_xg(team):
    avg_for = games.loc[games['home_team_name'] == team_stats[team]
    avg_for = avg_stoke.append(games.loc[games['away_team_name'] == team_stats['common_name'][0]])

SyntaxError: invalid syntax (<ipython-input-7-2d43a3413e47>, line 5)

In [ ]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [ ]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]



In [ ]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [ ]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [ ]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 

#Reading in stats

# Read in stats for new DF we'll use for testing

In [ ]:
test_prediction_games['home_xg'] = ""
test_prediction_games['away_xg'] = ""
test_prediction_games['home_xg_against'] = ""
test_prediction_games['away_xg_against'] = ""
test_prediction_games['home_total_corners'] = ""
test_prediction_games['away_total_corners'] = ""

# Creating new columns where our data will go

In [ ]:
def fill_stats_new_test(team):
    test_prediction_games['home_xg'] = np.where(test_prediction_games['home_team_name'] == team, xg_stats[team], test_prediction_games['home_xg'])
    test_prediction_games['away_xg'] = np.where(test_prediction_games['away_team_name'] == team, xg_stats[team], test_prediction_games['away_xg'])
    test_prediction_games['home_xg_against'] = np.where(test_prediction_games['home_team_name'] == team, xga_stats[team], test_prediction_games['home_xg_against'])
    test_prediction_games['away_xg_against'] = np.where(test_prediction_games['away_team_name'] == team, xga_stats[team], test_prediction_games['away_xg_against'])
    test_prediction_games['home_total_corners'] = np.where(test_prediction_games['home_team_name'] == team, corner_stats[team], test_prediction_games['home_total_corners'])
    test_prediction_games['away_total_corners'] = np.where(test_prediction_games['away_team_name'] == team, corner_stats[team], test_prediction_games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [ ]:
fill_stats_new_test('Fulham')
fill_stats_new_test('Blackburn Rovers')
fill_stats_new_test('West Bromwich Albion')
fill_stats_new_test('Queens Park Rangers')
fill_stats_new_test('AFC Bournemouth')
fill_stats_new_test('Middlesbrough')
fill_stats_new_test('Barnsley')
fill_stats_new_test('Millwall')
fill_stats_new_test('Sheffield United')
fill_stats_new_test('Reading')
fill_stats_new_test('Cardiff City')
fill_stats_new_test('Nottingham Forest')
fill_stats_new_test('Hull City')
fill_stats_new_test('Blackpool')
fill_stats_new_test('Luton Town')
fill_stats_new_test('Stoke City')
fill_stats_new_test('Swansea City')
fill_stats_new_test('Derby County')
fill_stats_new_test('Huddersfield Town')
fill_stats_new_test('Preston North End')
fill_stats_new_test('Coventry City')
fill_stats_new_test('Peterborough United')
fill_stats_new_test('Birmingham City')
fill_stats_new_test('Bristol City')

# Some more EDA

In [ ]:
games.corr()['over/under'].sort_values(ascending=False)[0:20]

# Now let's do some modeling

This will be a 'normal' model in which we use train_test_split to divide up the data

In [ ]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X = games[features]
y = games['over/under']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
logreg = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
preds = logreg.predict(X_test)

In [ ]:
accuracy_score(y_test, preds)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
preds = knn.predict(X_test)

In [ ]:
accuracy_score(y_test, preds)

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
preds = dt.predict(X_test)

In [ ]:
accuracy_score(y_test, preds)

In [ ]:
#decision trees

In [ ]:
y_test

Test the test set while it is trained on the original train set?

In [ ]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X = test_prediction_games[features]
y = test_prediction_games['over/under']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.9, random_state = 42)

In [ ]:
games[0:5]['home_team_name']

# Trying to set the train and test sets myself without train_test_split

# Predicting Over/Under 2.5

In [ ]:
# games = pd.read_csv('../footy_data/championship_games_2022.csv')
#Going to load this in again because I need the full dataset including incomplete games -- the ones I want to predict

In [ ]:
games.head()

In [ ]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X_train = games[0:72][features]
X_test = games[72:84][features]
y_train = games[0:72]['over/under']
y_test = games[72:84]['over/under']

In [ ]:
X_test

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
preds = logreg.predict(X_test)

In [ ]:
accuracy_score(y_test, preds)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
preds = knn.predict(X_test)

In [ ]:
accuracy_score(y_test, preds)

In [ ]:
preds

Next need to add something to track accuracy from week to week